In [15]:
!pip install torch transformers peft datasets

In [16]:
from datasets import load_dataset
dataset=load_dataset("SetFit/emotion")


Repo card metadata block was not found. Setting CardData to empty.


In [17]:
train_data=dataset["train"]
test_data=dataset["test"]

In [18]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")


In [19]:

# Tokenization function
def tokenize_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)

# Apply tokenization
train_dataset = train_data.map(tokenize_function, batched=True)
test_dataset = test_data.map(tokenize_function, batched=True)

# Convert to PyTorch format
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [20]:
from transformers import AutoModelForSequenceClassification

# Load BERT model (6 output classes)
base_model= AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
print(base_model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [22]:
from peft import LoraConfig,prepare_model_for_kbit_training,get_peft_model

lora_config=LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query","value"],
    lora_dropout=0.05,
)

base_model=prepare_model_for_kbit_training(base_model)
peft_model = get_peft_model(base_model, lora_config)

# Print trainable parameters
peft_model.print_trainable_parameters()



trainable params: 294,912 || all params: 109,781,766 || trainable%: 0.2686


In [23]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./model_checkpoints",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    load_best_model_at_end=True,
    fp16=True  # set False if CPU only
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer  # Tokenizer for processing text
)

trainer.train()


/tmp/ipython-input-1426907803.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,1.334800,1.218210
2,1.154600,1.093438
3,1.167500,1.056607


TrainOutput(global_step=3000, training_loss=1.2717955220540365, metrics={'train_runtime': 1003.7484, 'train_samples_per_second': 47.821, 'train_steps_per_second': 2.989, 'total_flos': 1.2673270775808e+16, 'train_loss': 1.2717955220540365, 'epoch': 3.0})

In [24]:
trainer.save_model("./my_emotion_model")
tokenizer.save_pretrained("./my_emotion_model")


('./my_emotion_model/tokenizer_config.json',
 './my_emotion_model/special_tokens_map.json',
 './my_emotion_model/vocab.txt',
 './my_emotion_model/added_tokens.json',
 './my_emotion_model/tokenizer.json')

In [25]:
from peft import PeftModel
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import pipeline

# Load base model
base_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)

# Load fine-tuned adapters
peft_model = PeftModel.from_pretrained(base_model, "./my_emotion_model")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("./my_emotion_model")

# Use pipeline
emotion_classifier = pipeline("text-classification", model=peft_model, tokenizer=tokenizer)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
The model 'PeftModel' is not supported for text-classification. Supported models are ['PeftModelForSequenceClassification', 'AlbertForSequenceClassification', 'ArceeForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequence

In [28]:
print(emotion_classifier("I am so happy today!"))
print(emotion_classifier("i ate i could feel a gentle tingle throughout almost as if i was feeling the healing taking place at a cellular level"))


[{'label': 'LABEL_1', 'score': 0.21514829993247986}]
[{'label': 'LABEL_1', 'score': 0.22266608476638794}]
